# IBM DATA SCIENCE CAPSTONE PROJECT
Eduardo Andre Cuya Nizama

## Libraries

In [2]:
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import geopy.geocoders
import folium

## Import data from week 3

In [3]:
data_toronto = pd.read_csv("TorontoData.csv")
data_toronto.drop(labels="Unnamed: 0",axis=1,inplace=True)
data_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Filtering Data

In [4]:
final_data = data_toronto[(data_toronto['Borough']=='Downtown Toronto') | 
               (data_toronto['Borough']=='East Toronto') | 
               (data_toronto['Borough']=='West Toronto')|
               (data_toronto['Borough']=='Central Toronto')|
              (data_toronto['Borough'] == 'Scarborough')]

final_data.reset_index(inplace=True)
final_data.drop('index',1,inplace=True)
final_data.head()

C:\Users\EDUARDOANDRECUYANIZA\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Ploting initial data 

In [5]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## Accessing Foursquare API

In [6]:
CLIENT_ID = 'SLRZJRDSKV4D1X2R4NLVBGWLPSMZH0OKQ2EVUUEMAOXKILST' # your Foursquare ID
CLIENT_SECRET = '1JYD1PLEET5TYIQ1LRSA30OF0VC35NTDURKJCCH5DBQKNZCW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SLRZJRDSKV4D1X2R4NLVBGWLPSMZH0OKQ2EVUUEMAOXKILST
CLIENT_SECRET:1JYD1PLEET5TYIQ1LRSA30OF0VC35NTDURKJCCH5DBQKNZCW


## Getting nearby Venues

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
LIMIT = 100
radius= 500
Toronto_venues = getNearbyVenues(names=final_data['Neighborhood'],
                                   latitudes=final_data['Latitude'],
                                   longitudes=final_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Domi

In [9]:
Toronto_venues.shape

(1791, 7)

In [10]:
venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
Toronto_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Business reply mail Processing Centre969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


## Filtering the top 10 venues for each Neighborhood

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Hotel,Bar,Restaurant,Bakery
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Steakhouse,Seafood Restaurant,Cheese Shop,Café,Italian Restaurant,Bakery
4,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## Making some clusters
Clusters created by similarity in top venues

In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ = np.append(kmeans.labels_,[2])
kmeans.labels_

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [14]:
Toronto_merged = final_data
Toronto_merged['Cluster Labels'] = kmeans.labels_
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head()


C:\Users\EDUARDOANDRECUYANIZA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Women's Store,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Breakfast Spot,Mexican Restaurant,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Soccer Field,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bakery,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## Plot map with clusters

In [15]:
# Create Map

toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finding the best candidate

First of all, we try to find in which Cluster was Roselawn

In [16]:
Toronto_merged[Toronto_merged['Neighborhood']=='Roselawn']

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Pool,Home Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant


Roselawn was in cluster 0, so the requirements for our candidates are being in the cluster 0 and in the Borough of Scarborough

In [17]:
Candidates = Toronto_merged.loc[(Toronto_merged['Cluster Labels'] == 0) & (Toronto_merged['Borough'] =='Scarborough')]
Candidates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Women's Store,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Soccer Field,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bakery,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Grocery Store,Spa,Playground,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [18]:
map_clusters = folium.Map(location=[toronto_latitude,toronto_longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Candidates['Latitude'], Candidates['Longitude'], Candidates['Neighborhood'], Candidates['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The other requirement from the client was to have grocery stores near

In [23]:
a=((Candidates=="Grocery Store"))
np.sum(a,axis=1)

0     0
1     0
3     0
4     0
5     1
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
dtype: int64

The only one candidate who satisfies this 3 conditions is candidate 5

In [19]:
search_query = 'Grocery Store'
radius = 5000
LIMIT = 10
latitude = Candidates.loc[5,'Latitude']
longitude = Candidates.loc[5,'Longitude']
print(latitude,longitude)

43.7447342 -79.23947609999998


In [20]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.sort_values(by='location.distance',inplace=True)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
2,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,50942b90e4b0a70261f30d18,1355 Danforth Road,CA,Scarborough,Canada,NaN,427,"[1355 Danforth Road, Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.74385420196717...",43.743854,-79.244650,NaN,NaN,ON,True Discount Convenience Store,v-1545668646
0,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",False,4d5c96f5e7f75481e5221c97,1351 Danforth Rd,CA,Toronto,Canada,NaN,428,"[1351 Danforth Rd, Toronto ON M1J 1G2, Canada]","[{'label': 'display', 'lat': 43.74378, 'lng': ...",43.743780,-79.244644,NaN,M1J 1G2,ON,Daisy Mart Convenience Store,v-1545668646
4,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4bb3e866f187a59379a013f8,2900 Eglinton Ave East,CA,Toronto,Canada,at McCowan Road,685,"[2900 Eglinton Ave East (at McCowan Road), Tor...","[{'label': 'display', 'lat': 43.73861453638707...",43.738615,-79.238576,NaN,M1J 2E4,ON,Mac's Convenience Store,v-1545668646
1,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",False,4b2405e2f964a5204b5f24e3,2727 Eglinton Ave E,CA,Scarborough,Canada,at Brimley Rd.,1233,"[2727 Eglinton Ave E (at Brimley Rd.), Scarbor...","[{'label': 'display', 'lat': 43.7359411, 'lng'...",43.735941,-79.248805,NaN,M1K 2S2,ON,The Beer Store,v-1545668646
9,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",False,4b677b90f964a520a6512be3,3205 Eglinton Ave E,CA,Scarborough,Canada,Markham Road,1715,"[3205 Eglinton Ave E (Markham Road), Scarborou...","[{'label': 'display', 'lat': 43.7433482, 'lng'...",43.743348,-79.218232,NaN,M1J 2H7,ON,The Beer Store,v-1545668646
7,"[{'id': '4bf58dd8d48988d101951735', 'name': 'T...",False,4bfebef9e584c928402b6d25,1225 Kennedy Rd,CA,Scarborough,Canada,NaN,3199,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]","[{'label': 'display', 'lat': 43.75578210550947...",43.755782,-79.276208,NaN,M1P 2L4,ON,Salvation Army Thrift Store,v-1545668646
3,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",False,5085ec39e4b0b1ead2eb0818,300 Borough Drive,CA,Scarborough,Canada,in Scarborough Town Centre,3702,[300 Borough Drive (in Scarborough Town Centre...,"[{'label': 'display', 'lat': 43.775537, 'lng':...",43.775537,-79.256833,NaN,M1P 4P5,ON,Disney Store,v-1545668646
6,"[{'id': '52f2ab2ebcbc57f1066b8b36', 'name': 'I...",False,4bc33a9f4cdfc9b626029721,300 Borough Dr UNIT 95,CA,Scarborough,Canada,NaN,3743,"[300 Borough Dr UNIT 95, Scarborough ON M1P 4...","[{'label': 'display', 'lat': 43.77564497500000...",43.775645,-79.257800,NaN,M1P 4P5,ON,TELUS Store,v-1545668646
5,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,4d1127d65f3376eb86661386,300 Borough Dr,CA,Scarborough,Canada,NaN,3767,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458...",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,Tommy Hilfiger Company Store,v-1545668646
8,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4dd19e051838a75196486487,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,5162,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto O...","[{'label': 'display', 'lat': 43.74331671353403...",43.743317,-79.303645,NaN,M1R 2Y3,ON,Ghadir Mid-Eastern Grocery,v-1545668646


Finally, we analize the popularity of the groceries in a radius of 500 for the candidate chosen

In [21]:
def rate(id):
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

In [22]:
dataframe['id'].apply(lambda id: rate(id) )

This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
6.3
6.2
This venue has not been rated yet.
7.7
This venue has not been rated yet.
7.4
8.1


2    None
0    None
4    None
1    None
9    None
7    None
3    None
6    None
5    None
8    None
Name: id, dtype: object

# The best candidate 
Scarborough Village

In [30]:
ScarboroughVillage_latitude = 43.744734; ScarboroughVillage_longitude = -79.239476
map_ScarboroughVillage = folium.Map(location = [ScarboroughVillage_latitude, ScarboroughVillage_longitude], zoom_start = 10)
folium.CircleMarker(
    [ScarboroughVillage_latitude, ScarboroughVillage_longitude],
    radius=5,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7).add_to(map_ScarboroughVillage)      
map_ScarboroughVillage